#### from_json

- Function is used to convert **JSON string** into **Struct type or Map type**.
- If the **string** is **unparseable**, it returns **null**.

**Pattern**
- Define **schema** (StructType / ArrayType)
- Apply **from_json(col("json_string"), schema)**
- Extract **struct/array** fields

#### Syntax

     from_json(col, schema, options={})


**options:**
**allowUnquotedFieldNames:**
- If set to true, allows unquoted field names in the JSON string. Default is false.

**allowSingleQuotes:**
- If set to true, allows single quotes instead of double quotes in the JSON string. Default is true.

**allowNumericLeadingZero:**
- If set to true, allows leading zeros in numeric values. Default is false.

**allowBackslashEscapingAnyCharacter:**
- If set to true, allows backslash escaping any character in the JSON string. Default is false.

**allowUnquotedControlChars:**
- If set to true, allows unquoted control characters in the JSON string. Default is false.

**mode:**
- Specifies the parsing mode. It can be one of the following values:

- **PERMISSIVE:**
  - Tries to parse all JSON records and sets fields to null if parsing fails. This is the default mode.

- **DROPMALFORMED:**
  - Drops the whole row if any parsing error occurs.

- **FAILFAST:**
  - Fails immediately if any parsing error occurs.

**What’s json format?**
- The json format is meaning, **JavaScript Object Notation**.

  - Data is in **name, value** pairs.
  - Data is separated by **commas**.
  - **Curly braces** hold **objects**.
  - **Square brackets** hold **arrays**.

In [0]:
from pyspark.sql.functions import from_json, col, explode
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, IntegerType, BooleanType, DoubleType

##### 1) Basic Usage with StructType
- Convert **JSON string** column into **struct**.

In [0]:
data = [
    ('{"name": "Albert", "age": 30, "city": "Bangalore"}',),
    ('{"name": "Bobby", "age": 25, "city": "Chennai"}',),
    ('{"name": "Swapna", "age": 35, "city": "Hyderabad"}',),
    ('{"name": "David", "age": 28, "city": "Cochin"}',),
    ('{"name": "Anand", "age": 33, "city": "Baroda"}',),
    ('{"name": "Baskar", "age": 29, "city": "Nasik"}',),
]

df_bjs = spark.createDataFrame(data, ["json_string"])

schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("city", StringType(), True)
])

df_bjs_json = df_bjs.withColumn("json_data", from_json(col("json_string"), schema))
display(df_bjs_json)

json_string,json_data
"{""name"": ""Albert"", ""age"": 30, ""city"": ""Bangalore""}","List(Albert, 30, Bangalore)"
"{""name"": ""Bobby"", ""age"": 25, ""city"": ""Chennai""}","List(Bobby, 25, Chennai)"
"{""name"": ""Swapna"", ""age"": 35, ""city"": ""Hyderabad""}","List(Swapna, 35, Hyderabad)"
"{""name"": ""David"", ""age"": 28, ""city"": ""Cochin""}","List(David, 28, Cochin)"
"{""name"": ""Anand"", ""age"": 33, ""city"": ""Baroda""}","List(Anand, 33, Baroda)"
"{""name"": ""Baskar"", ""age"": 29, ""city"": ""Nasik""}","List(Baskar, 29, Nasik)"


**Extract Struct Fields**

     df_extracted = df_bjs_json.select(col("json_data.name").alias("name"),
                                       col("json_data.age").alias("age"),
                                       col("json_data.city").alias("city")
                                      )
                                      (or)
     df.select("json_data.*").display()

In [0]:
df_bjs_json.select("json_data.*").display()

name,age,city
Albert,30,Bangalore
Bobby,25,Chennai
Swapna,35,Hyderabad
David,28,Cochin
Anand,33,Baroda
Baskar,29,Nasik


In [0]:
df_extracted = df_bjs_json.select(
    col("json_data.name").alias("name"),
    col("json_data.age").alias("age"),
    col("json_data.city").alias("city")
)
df_extracted.display()

name,age,city
Albert,30,Bangalore
Bobby,25,Chennai
Swapna,35,Hyderabad
David,28,Cochin
Anand,33,Baroda
Baskar,29,Nasik


##### 2) Nested JSON

In [0]:
# nested JSON rows
nested_data = [
    ('{"id":1, "Name":"Jyoti", "info":{"Experience":8, "city":"Bangalore","pin":560001}}',),
    ('{"id":2, "Name":"Sooraj", "info":{"Experience":10, "city":"Mysore","pin":570001}}',),
    ('{"id":3, "Name":"Swapna", "info":{"Experience":6, "city":"Chennai","pin":600001}}',),
    ('{"id":4, "Name":"Ravi", "info":{"Experience":5, "city":"Hyderabad","pin":500001}}',),
    ('{"id":5, "Name":"Anupam", "info":{"Experience":3, "city":"Mumbai","pin":400001}}',),
    ('{"id":6, "Name":"Kedar", "info":{"Experience":11, "city":"Delhi","pin":110001}}',),
    ('{"id":7, "Name":"Vivek", "info":{"Experience":9, "city":"Kolkata","pin":700001}}',),
    ('{"id":8, "Name":"Nagendar", "info":{"Experience":12, "city":"Pune","pin":411001}}',),
    ('{"id":9, "Name":"Madhu", "info":{"Experience":4, "city":"Ahmedabad","pin":380001}}',),
    ('{"id":10, "Name":"Sanjana", "info":{"Experience":2, "city":"Jaipur","pin":302001}}',),
]

# Create DataFrame with JSON string column
df_nested = spark.createDataFrame(nested_data, ["json_string"])

# Define schema for nested JSON
nested_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("info", StructType([
        StructField("Experience", IntegerType(), True),
        StructField("city", StringType(), True),
        StructField("pin", IntegerType(), True),
    ]))
])

# Parse JSON into struct
df_nested_parsed = df_nested.withColumn("json_data", from_json(col("json_string"), nested_schema))
display(df_nested_parsed)

json_string,json_data
"{""id"":1, ""Name"":""Jyoti"", ""info"":{""Experience"":8, ""city"":""Bangalore"",""pin"":560001}}","List(1, Jyoti, List(8, Bangalore, 560001))"
"{""id"":2, ""Name"":""Sooraj"", ""info"":{""Experience"":10, ""city"":""Mysore"",""pin"":570001}}","List(2, Sooraj, List(10, Mysore, 570001))"
"{""id"":3, ""Name"":""Swapna"", ""info"":{""Experience"":6, ""city"":""Chennai"",""pin"":600001}}","List(3, Swapna, List(6, Chennai, 600001))"
"{""id"":4, ""Name"":""Ravi"", ""info"":{""Experience"":5, ""city"":""Hyderabad"",""pin"":500001}}","List(4, Ravi, List(5, Hyderabad, 500001))"
"{""id"":5, ""Name"":""Anupam"", ""info"":{""Experience"":3, ""city"":""Mumbai"",""pin"":400001}}","List(5, Anupam, List(3, Mumbai, 400001))"
"{""id"":6, ""Name"":""Kedar"", ""info"":{""Experience"":11, ""city"":""Delhi"",""pin"":110001}}","List(6, Kedar, List(11, Delhi, 110001))"
"{""id"":7, ""Name"":""Vivek"", ""info"":{""Experience"":9, ""city"":""Kolkata"",""pin"":700001}}","List(7, Vivek, List(9, Kolkata, 700001))"
"{""id"":8, ""Name"":""Nagendar"", ""info"":{""Experience"":12, ""city"":""Pune"",""pin"":411001}}","List(8, Nagendar, List(12, Pune, 411001))"
"{""id"":9, ""Name"":""Madhu"", ""info"":{""Experience"":4, ""city"":""Ahmedabad"",""pin"":380001}}","List(9, Madhu, List(4, Ahmedabad, 380001))"
"{""id"":10, ""Name"":""Sanjana"", ""info"":{""Experience"":2, ""city"":""Jaipur"",""pin"":302001}}","List(10, Sanjana, List(2, Jaipur, 302001))"


In [0]:
df_nested_parsed.select(
    col("json_data.id").alias("Id"),
    col("json_data.Name").alias("Name"),
    col("json_data.info.Experience").alias("Experience"),
    col("json_data.info.city").alias("City"),
    col("json_data.info.pin").alias("Pin")
).display()

Id,Name,Experience,City,Pin
1,Jyoti,8,Bangalore,560001
2,Sooraj,10,Mysore,570001
3,Swapna,6,Chennai,600001
4,Ravi,5,Hyderabad,500001
5,Anupam,3,Mumbai,400001
6,Kedar,11,Delhi,110001
7,Vivek,9,Kolkata,700001
8,Nagendar,12,Pune,411001
9,Madhu,4,Ahmedabad,380001
10,Sanjana,2,Jaipur,302001


##### 3) Array of JSON Objects

In [0]:
array_data = [
    ('[{"name":"Alice","age":25},{"name":"Bob","age":30}]',),
    ('[{"name":"Charlie","age":28},{"name":"David","age":35}]',),
    ('[{"name":"Eve","age":22},{"name":"Frank","age":40}]',),
    ('[{"name":"Grace","age":27},{"name":"Heidi","age":33}]',),
    ('[{"name":"Ivan","age":29},{"name":"Judy","age":31}]',),
    ('[{"name":"Kevin","age":26},{"name":"Laura","age":34}]',),
    ('[{"name":"Mallory","age":38},{"name":"Niaj","age":32}]',),
    ('[{"name":"Olivia","age":24},{"name":"Peggy","age":36}]',),
    ('[{"name":"Quentin","age":37},{"name":"Ruth","age":23}]',),
    ('[{"name":"Sybil","age":39},{"name":"Trent","age":28}]',),
]

df_array = spark.createDataFrame(array_data, ["json_string"])

array_schema = ArrayType(
    StructType([
        StructField("name", StringType(), True),
        StructField("age", IntegerType(), True)
    ])
)

# Parse JSON string into array<struct>
df_array_parsed = df_array.withColumn("json_data", from_json(col("json_string"), array_schema))
display(df_array_parsed)

json_string,json_data
"[{""name"":""Alice"",""age"":25},{""name"":""Bob"",""age"":30}]","List(List(Alice, 25), List(Bob, 30))"
"[{""name"":""Charlie"",""age"":28},{""name"":""David"",""age"":35}]","List(List(Charlie, 28), List(David, 35))"
"[{""name"":""Eve"",""age"":22},{""name"":""Frank"",""age"":40}]","List(List(Eve, 22), List(Frank, 40))"
"[{""name"":""Grace"",""age"":27},{""name"":""Heidi"",""age"":33}]","List(List(Grace, 27), List(Heidi, 33))"
"[{""name"":""Ivan"",""age"":29},{""name"":""Judy"",""age"":31}]","List(List(Ivan, 29), List(Judy, 31))"
"[{""name"":""Kevin"",""age"":26},{""name"":""Laura"",""age"":34}]","List(List(Kevin, 26), List(Laura, 34))"
"[{""name"":""Mallory"",""age"":38},{""name"":""Niaj"",""age"":32}]","List(List(Mallory, 38), List(Niaj, 32))"
"[{""name"":""Olivia"",""age"":24},{""name"":""Peggy"",""age"":36}]","List(List(Olivia, 24), List(Peggy, 36))"
"[{""name"":""Quentin"",""age"":37},{""name"":""Ruth"",""age"":23}]","List(List(Quentin, 37), List(Ruth, 23))"
"[{""name"":""Sybil"",""age"":39},{""name"":""Trent"",""age"":28}]","List(List(Sybil, 39), List(Trent, 28))"


In [0]:
# Explode array into individual rows
df_final = df_array_parsed.withColumn("person", explode(col("json_data"))) \
    .select("person",
        col("person.name").alias("name"),
        col("person.age").alias("age")
    )

display(df_final)

person,name,age
"List(Alice, 25)",Alice,25
"List(Bob, 30)",Bob,30
"List(Charlie, 28)",Charlie,28
"List(David, 35)",David,35
"List(Eve, 22)",Eve,22
"List(Frank, 40)",Frank,40
"List(Grace, 27)",Grace,27
"List(Heidi, 33)",Heidi,33
"List(Ivan, 29)",Ivan,29
"List(Judy, 31)",Judy,31


##### 4) Using from_json with Maps

In [0]:
from pyspark.sql.types import MapType

data_map = [
    ('{"id":1,"scores":{"math":90,"science":85}}',),
    ('{"id":2,"scores":{"math":75,"science":95}}',),
    ('{"id":3,"scores":{"math":80,"science":88}}',),
    ('{"id":4,"scores":{"math":85,"science":91}}',),
    ('{"id":5,"scores":{"math":79,"science":82}}',),
    ('{"id":6,"scores":{"math":83,"science":92}}',)
]

df_map = spark.createDataFrame(data_map, ["json_string"])

schema_map = StructType([
    StructField("id", IntegerType(), True),
    StructField("scores", MapType(StringType(), IntegerType()), True)
])

df_map_parsed = df_map.withColumn("json_data", from_json(col("json_string"), schema_map))
display(df_map_parsed)

json_string,json_data
"{""id"":1,""scores"":{""math"":90,""science"":85}}","List(1, Map(math -> 90, science -> 85))"
"{""id"":2,""scores"":{""math"":75,""science"":95}}","List(2, Map(math -> 75, science -> 95))"
"{""id"":3,""scores"":{""math"":80,""science"":88}}","List(3, Map(math -> 80, science -> 88))"
"{""id"":4,""scores"":{""math"":85,""science"":91}}","List(4, Map(math -> 85, science -> 91))"
"{""id"":5,""scores"":{""math"":79,""science"":82}}","List(5, Map(math -> 79, science -> 82))"
"{""id"":6,""scores"":{""math"":83,""science"":92}}","List(6, Map(math -> 83, science -> 92))"


In [0]:
df_map_parsed.select("json_data.id", "json_data.scores").display()

id,scores
1,"Map(math -> 90, science -> 85)"
2,"Map(math -> 75, science -> 95)"
3,"Map(math -> 80, science -> 88)"
4,"Map(math -> 85, science -> 91)"
5,"Map(math -> 79, science -> 82)"
6,"Map(math -> 83, science -> 92)"


##### 5) nested_json_string

In [0]:
# JSON string
nested_json_string = '''{
    "name": "Alice",
    "age": 28,
    "address": {
        "street": "123 Main St",
        "city": "San Francisco",
        "zip": "94105"
    }
}'''

# Create DataFrame with JSON string
df_string = spark.createDataFrame([(nested_json_string,)], ["json_string"])

# Define schema for nested JSON
schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("address", StructType([
        StructField("street", StringType(), True),
        StructField("city", StringType(), True),
        StructField("zip", StringType(), True)
    ]), True)
])

# Parse JSON using from_json
df_parsed = df_string.withColumn("json_data", from_json(col("json_string"), schema))

print("Parsed DataFrame:")
display(df_parsed)

# Access nested fields
df_selected = df_parsed.select(
    col("json_data.name").alias("Name"),
    col("json_data.age").alias("Age"),
    col("json_data.address.street").alias("Street"),
    col("json_data.address.city").alias("City"),
    col("json_data.address.zip").alias("ZipCode")
)

print("Flattened DataFrame:")
display(df_selected)

Parsed DataFrame:


json_string,json_data
"{ ""name"": ""Alice"", ""age"": 28, ""address"": { ""street"": ""123 Main St"", ""city"": ""San Francisco"", ""zip"": ""94105"" } }","List(Alice, 28, List(123 Main St, San Francisco, 94105))"


Flattened DataFrame:


Name,Age,Street,City,ZipCode
Alice,28,123 Main St,San Francisco,94105


##### 6) Error Handling (Invalid JSON)

- When **JSON strings** are **invalid**, from_json returns **null**.

In [0]:
# Define the JSON schema
json_schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", StringType(), True)
])

data_invalid = [
    ('{"name":"Joseph","age":25}',),
    ('{"name":"Bharath","age":}',),  # invalid JSON
    ('{"name":"Jagadish","age":35}',),
    ('{"name":"Somu","age":28}',),
    ('{"name":"Dhamu","age":22}',),
    ('{"name":"Bharath","age":}',),  # invalid JSON
]
df_invalid = spark.createDataFrame(data_invalid, ["json_string"])

df_invalid_parsed = df_invalid.withColumn("json_data", from_json(col("json_string"), json_schema))
display(df_invalid_parsed)

json_string,json_data
"{""name"":""Joseph"",""age"":25}","List(Joseph, 25)"
"{""name"":""Bharath"",""age"":}","List(null, null)"
"{""name"":""Jagadish"",""age"":35}","List(Jagadish, 35)"
"{""name"":""Somu"",""age"":28}","List(Somu, 28)"
"{""name"":""Dhamu"",""age"":22}","List(Dhamu, 22)"
"{""name"":""Bharath"",""age"":}","List(null, null)"


##### 7) Handling missing or corrupt data

**PERMISSIVE**

- **PERMISSIVE** is the **default mode** in Spark.
- Whether you **explicitly** set **{"mode": "PERMISSIVE"} or omit it**, the behavior is the **same**:
  - **Malformed rows** → parsed as **null**.
  - **Missing fields** → parsed as **null** in the **struct**.

In [0]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType

# Sample DataFrame with JSON strings
df_dropMal = spark.createDataFrame([
    (1, '{"name": "Ramesh", "age": 30, "isStudent": false, "score": 9.5, "address": null}'),  # valid
    (2, '{"name": "Johny", "age": 25, "isStudent": true}'),                                   # valid but missing fields
    (3, '{"name": "Baskar", "age": 40, "isStudent": true')                                    # ❌ malformed JSON
], ["id", "data"])

# Define schema for the JSON column
schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("isStudent", BooleanType(), True),
    StructField("score", DoubleType(), True),
    StructField("address", StringType(), True)
])

# Parse JSON using Permissive mode
df_permissive = df_dropMal.withColumn(
    "jsonData",
    from_json(col("data"), schema, {"mode": "PERMISSIVE"})
)

display(df_permissive)

# Flatten parsed struct into individual columns
df_flattened = df_permissive.select(
    col("id"),
    col("jsonData.name").alias("Name"),
    col("jsonData.age").alias("Age"),
    col("jsonData.isStudent").alias("IsStudent"),
    col("jsonData.score").alias("Score"),
    col("jsonData.address").alias("Address")
)

display(df_flattened)

id,data,jsonData
1,"{""name"": ""Ramesh"", ""age"": 30, ""isStudent"": false, ""score"": 9.5, ""address"": null}","List(Ramesh, 30, false, 9.5, null)"
2,"{""name"": ""Johny"", ""age"": 25, ""isStudent"": true}","List(Johny, 25, true, null, null)"
3,"{""name"": ""Baskar"", ""age"": 40, ""isStudent"": true","List(null, null, null, null, null)"


id,Name,Age,IsStudent,Score,Address
1,Ramesh,30,false,9.5,null
2,Johny,25,true,null,null
3,null,null,null,null,null


**FAILFAST**

- **FAILFAST** means Spark will immediately **throw an error** if it encounters **malformed JSON**.
- If JSON is **valid** but **some fields are missing**, it will not fail — missing fields just become **null**.

In [0]:
# Sample DataFrame with one malformed JSON row
df_failfast = spark.createDataFrame([
    (1, '{"name": "John", "age": 30, "isStudent": false, "score": 9.5, "address": null}'),  # valid
    (2, '{"name": "Jane", "age": 25, "isStudent": true}'),                                  # valid (missing fields -> nulls)
    (3, '{"name": "Bob", "age": 40, "isStudent": true')                                     # ❌ malformed (missing closing brace)
], ["id", "data"])


# Define schema for the JSON column
schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("isStudent", BooleanType(), True),
    StructField("score", DoubleType(), True),
    StructField("address", StringType(), True)
])

# Parse JSON using FAILFAST mode (throws error on malformed JSON)
df_failfast_parsed = df_failfast.withColumn(
    "jsonData",
    from_json(col("data"), schema, {"mode": "FAILFAST"})
)

display(df_failfast_parsed)   # ❌ Will throw error on malformed JSON

---------------------------------------------------------------------------
SparkException                            Traceback (most recent call last)
File <command-6396222873385835>, line 24
     18 # Parse JSON using FAILFAST mode (throws error on malformed JSON)
     19 df_failfast_parsed = df_failfast.withColumn(
     20     "jsonData",
     21     from_json(col("data"), schema, {"mode": "FAILFAST"})
     22 )
---> 24 display(df_failfast_parsed)

File /databricks/python_shell/lib/dbruntime/display.py:133, in Display.display(self, input, *args, **kwargs)
    131     pass
    132 elif self._cf_helper is not None and isinstance(input, ConnectDataFrame):
--> 133     self.display_connect_table(input, **kwargs)
    134 elif isinstance(input, ConnectDataFrame):
    135     if input.isStreaming:

File /databricks/python_shell/lib/dbruntime/display.py:97, in Display.display_connect_table(self, df, **kwargs)
     94     self.cf_helper.display_streaming_dataframe(df, config, self.streaming_l

**DropMalformed**
- **Drops** the **whole row** if any **parsing error** occurs.
- DROPMALFORMED **drops malformed** JSON rows (i.e., sets parsed column to null).
- **Missing fields do not count** as malformed → they will just be **null** in the parsed struct.

In [0]:
# Sample DataFrame with JSON strings
df_dropMal = spark.createDataFrame([
    (1, '{"name": "John", "age": 30, "isStudent": false, "score": 9.5, "address": null}'),  # valid
    (2, '{"name": "Jane", "age": 25, "isStudent": true}'),                                  # valid but missing fields
    (3, '{"name": "Bob", "age": 40, "isStudent": true')                                     # ❌ malformed JSON
], ["id", "data"])

display(df_dropMal)

# Define schema for the JSON column
schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("isStudent", BooleanType(), True),
    StructField("score", DoubleType(), True),
    StructField("address", StringType(), True)
])

# Parse JSON using DropMalformed mode
df_dropMalformed = df_dropMal.withColumn(
    "jsonData",
    from_json(col("data"), schema, {"mode": "DROPMALFORMED"})
)

display(df_dropMalformed)

# Flatten parsed struct into individual columns
df_flattened = df_dropMalformed.select(
    col("id"),
    col("jsonData.name").alias("Name"),
    col("jsonData.age").alias("Age"),
    col("jsonData.isStudent").alias("IsStudent"),
    col("jsonData.score").alias("Score"),
    col("jsonData.address").alias("Address")
)

display(df_flattened)

id,data
1,"{""name"": ""John"", ""age"": 30, ""isStudent"": false, ""score"": 9.5, ""address"": null}"
2,"{""name"": ""Jane"", ""age"": 25, ""isStudent"": true}"
3,"{""name"": ""Bob"", ""age"": 40, ""isStudent"": true"


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7200196016769780>, line 30
     24 # Parse JSON using DropMalformed mode
     25 df_dropMalformed = df_dropMal.withColumn(
     26     "jsonData",
     27     from_json(col("data"), schema, {"mode": "DROPMALFORMED"})
     28 )
---> 30 display(df_dropMalformed)
     32 # Flatten parsed struct into individual columns
     33 df_flattened = df_dropMalformed.select(
     34     col("id"),
     35     col("jsonData.name").alias("Name"),
   (...)
     39     col("jsonData.address").alias("Address")
     40 )

File /databricks/python_shell/lib/dbruntime/display.py:133, in Display.display(self, input, *args, **kwargs)
    131     pass
    132 elif self._cf_helper is not None and isinstance(input, ConnectDataFrame):
--> 133     self.display_connect_table(input, **kwargs)
    134 elif isinstance(input, ConnectDataFrame):
    135    